 # 1. DEFINE I/O DATA FRAMES

In [1]:
import numpy as np 
import pandas as pd 

MODEL_NAME='dblstm_crf'
HPARAMS_STR='lr5.00e-04_hs512x256x128x64_ctx-p3_wang2vec_s100'
MODEL_V= '00'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'evaluation_valid.csv')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/dblstm_crf/lr5.00e-04_hs512x256x128x64_ctx-p3_wang2vec_s100/00/Yhat_valid.csv


In [2]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 17) (12298, 2)


In [3]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 19)


In [4]:
df.to_csv(TARGET_PATH)

In [5]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,FUNC,M_R,CTX_P-3,CTX_P-2,CTX_P-1,CTX_P+1,CTX_P+2,CTX_P+3,ARG_0,ARG_1,Y_0,Y_1
IDX,,,,,,,,,,,,,,,,,,,
123846,1,2879,5100,0,Entre,entre,estar,-,0,entre,o,convidado,o,-,rei,(A2*,A2,(A2*,A2
123847,2,2879,5100,0,os,o,estar,-,0,entre,o,convidado,o,-,rei,*,A2,*,A2
123848,3,2879,5100,0,convidados,convidado,estar,-,0,entre,o,convidado,o,-,rei,*),A2,*),A2
123849,4,2879,5100,0,estão,estar,estar,estar,1,entre,o,convidado,o,-,rei,(V*),V,(V*),V
123850,5,2879,5100,0,o,o,estar,-,1,entre,o,convidado,o,-,rei,(A1*,A1,(A1*,A1


In [6]:
df2= df[['ARG_1','Y_1']]
table= pd.pivot_table(df2.iloc[:30,:], index='ARG_1', columns='Y_1', aggfunc=len, fill_value=0)
table

Y_1,*,A0,A1,A2,V
ARG_1,,,,,
*,1,0,0,0,0
A0,0,1,0,0,0
A1,0,0,24,0,0
A2,0,0,0,3,0
V,0,0,0,0,1


 # 2. CONFUSION MATRIX

In [7]:
df1= df[['ARG_1','Y_1']]
table= pd.pivot_table(df1, index='ARG_1', columns='Y_1', aggfunc=len, fill_value=0)

#Makes a unique index out of ground truth and predictions, filling with zeros
new_index= table.index.union(table.columns)
table= table.reindex(new_index, axis=0, fill_value=0)
table= table.reindex(new_index, axis=1, fill_value=0)
table.columns.names=['predictions']
table.index.name='ground truth'
table


ground truth,*,A0,A1,A2,A3,A4,AM-ADV,AM-CAU,AM-DIR,AM-DIS,...,C-A0,C-A1,C-A2,C-AM-EXT,C-AM-MNR,C-AM-NEG,C-AM-PRD,C-AM-TMP,C-V,V
ground truth,,,,,,,,,,,,,,,,,,,,,
*,4183,101,436,183,0,2,30,50,0,9,...,0,0,0,0,0,0,0,0,2,2
A0,281,421,107,34,0,0,1,0,0,9,...,0,2,0,0,0,0,0,0,2,0
A1,477,49,2073,166,6,0,0,30,0,2,...,0,2,0,0,0,0,0,0,1,4
A2,276,8,321,328,0,1,3,47,0,0,...,0,2,19,0,0,0,0,0,2,1
A3,20,4,17,22,3,4,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
A4,4,0,6,10,0,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AM-ADV,61,2,15,16,0,0,7,0,0,2,...,0,0,0,0,0,0,0,0,0,0
AM-CAU,84,9,1,27,0,0,0,17,0,0,...,0,0,0,0,0,0,1,0,0,0
AM-DIR,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 ## 3 Evaluation
 ### Precision .: How many selected items are relevant?
 ### Recall .: How many relevant items are selected?

In [8]:
# BASE MATRICES
diag = np.diag(table.as_matrix()).astype(np.float32)
retrieved= np.sum(table.as_matrix(), axis=0, keepdims=False).astype(np.float32) #preserve columns, sum over rows
relevant= np.sum(table.as_matrix(), axis=1, keepdims=False).astype(np.float32) #preserve rows, sum over columns

#PRECISION
P=np.nan_to_num(diag / retrieved)
precisiondf= pd.DataFrame(data= P, index=table.index, columns=['precision']).fillna(0)
precisiondf.index.name= 'ARG'
# precisiondf

#RECALL
R= np.nan_to_num(diag / relevant)
recalldf= pd.DataFrame(data= R, index=table.index, columns=['recall']).fillna(0)
recalldf.index.name= 'ARG'
# recalldf

#F1
F1= np.nan_to_num(2* ((P * R)/(P + R)))
f1scoredf= pd.DataFrame(data= F1, index=table.index, columns=['f1 score']).fillna(0)
f1scoredf.index.name= 'ARG'

#Concatenate
evaluationdf= pd.concat((precisiondf, recalldf, f1scoredf), axis=1)
evaluationdf








/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/Users/Varela/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


,precision,recall,f1 score
ARG,,,
*,0.724705,0.814447,0.766960
A0,0.684553,0.462130,0.551769
A1,0.629135,0.712616,0.668279
A2,0.364444,0.296833,0.327182
A3,0.250000,0.036145,0.063158
A4,0.411765,0.155556,0.225806
AM-ADV,0.148936,0.064220,0.089744
AM-CAU,0.106918,0.111111,0.108974
AM-DIR,0.000000,0.000000,0.000000


## 3.1 Total scores 

In [9]:
df2= df1[['ARG_1']]
df2['N_ARGS']=df2.groupby('ARG_1')['ARG_1'].transform('size')
df2.drop_duplicates(subset='ARG_1', inplace=True)
# df2
df2.set_index('ARG_1', inplace=True)
df2=df2.reindex(index=new_index, fill_value=0)
df2
# Get the weights for each
weights= df2['N_ARGS'].as_matrix()

#Totals
#precision
p=100*P.dot(weights)/np.sum(weights)
print('precision: {:.2f}\%'.format(p))
#recall
r=100*R.dot(weights)/np.sum(weights)
print('recall: {:.2f}\%'.format(r))
#F1 score
print('f1 score: {:.2f}\%'.format(2*(p*r)/(p+r)))



precision: 61.16\%
recall: 63.83\%
f1 score: 62.47\%


In [13]:
#perfect prop
df['ERR'] = 1
ind = df['ARG_1'] == df['Y_1']
df.loc[ind,('ERR')]=0

table= pd.pivot_table(df[['P', 'ERR']], index=['P'], columns=['ERR'], aggfunc=np.sum)
table


P
